# Sachsen

Every federal state is represented by its own input directory and is processed into a NUTS level 2 directory containing a sub-folder for each discharge location. These folder names are derived from NUTS and reflect the CAMELS id. The NUTS level 2 code for Sachsen is `DED`.

To pre-process the data, you need to write (at least) two functions. One should extract all metadata and condense it into a single `pandas.DataFrame`. This is used to build the folder structure and derive the ids.
The second function has to take an id, as provided by the state authorities, called `provider_id` and return a `pandas.DataFrame` with the transformed data. The dataframe needs the three columns `['date', 'q' | 'w', 'flag']`.

For easier and unified output handling, the `camelsp` package contains a context object called `Bundesland`. It takes a number of names and abbreviations to identify the correct federal state and returns an object that holds helper and save functions.

The context saves files as needed and can easily be changed to save files with different strategies, ie. fill missing data with NaN, merge data into a single file, create files for each variable or pack everything together into a netcdf.

In [1]:
import pandas as pd
import numpy as np
from pandas.errors import ParserError
import os
from glob import glob
from pprint import pprint
from tqdm import tqdm
from typing import Union, Dict
from datetime import datetime as dt
from dateparser import parse
import warnings

from camelsp import Bundesland

The context can also be instantiated as any regular Python class, ie. to load only the default input data path, that we will user later.

In [2]:
# the context also makes the input path available, if camelsp was install locally
BASE = Bundesland('Sachsen').input_path
BASE

'/home/alexander/Github/camels/camelsp/input_data/SN_Sachsen'

## Parse data

We do not have a Metadata file, but one Excel file for each station. Thus we need to parse each metadata individually and collect.

In [3]:
files = glob(os.path.join(BASE, '*.xlsx'))
print(f"Found {len(files)} files.")

Found 282 files.


Test for the first file:

In [4]:
with warnings.catch_warnings():
    df = pd.read_excel(files[32], skiprows=2, decimal=',')


print(f"Pegelkennziffer: {df.Pegelkennziffer.unique()}")
print(f"Pegelname:       {df.Pegelname.unique()}")
print(f"Gewaesser:       {df.Gewaesser.unique()}")
print(f"Beeinflussung:   {df.Beeinflussung.unique()}")
print(f"Datum type:      {df.Datum.dtype}")

df

/home/alexander/anaconda3/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Pegelkennziffer: [530020]
Pegelname:       ['Klingenthal 1']
Gewaesser:       ['Zwota (Svatava)']
Beeinflussung:   [nan 'R']
Datum type:      datetime64[ns]


,Pegelkennziffer,Pegelname,Gewaesser,Datum,Wasserstand (W) cm,Durchfluss (Q) m³/s,Beeinflussung
0,530020,Klingenthal 1,Zwota (Svatava),1961-01-01,NaN,0.880,NaN
1,530020,Klingenthal 1,Zwota (Svatava),1961-01-02,NaN,0.880,NaN
2,530020,Klingenthal 1,Zwota (Svatava),1961-01-03,NaN,0.880,NaN
3,530020,Klingenthal 1,Zwota (Svatava),1961-01-04,NaN,0.880,NaN
4,530020,Klingenthal 1,Zwota (Svatava),1961-01-05,NaN,0.880,NaN
...,...,...,...,...,...,...,...
21545,530020,Klingenthal 1,Zwota (Svatava),2020-12-27,33.0,1.070,NaN
21546,530020,Klingenthal 1,Zwota (Svatava),2020-12-28,34.0,1.140,NaN
21547,530020,Klingenthal 1,Zwota (Svatava),2020-12-29,33.0,1.060,NaN
21548,530020,Klingenthal 1,Zwota (Svatava),2020-12-30,32.0,0.995,NaN


One question, what does Beeinflussung actually mean here? Ignoring it for now.

Go for each file and extract metadata and the two data columns

In [5]:
np.unique(df[df.columns[1]].values)

array(['Klingenthal 1'], dtype=object)

In [6]:
# create result container
meta = []
q = []
w = []

with warnings.catch_warnings(record=True) as warns:
    for filename in tqdm(files):
        # read
        with warnings.catch_warnings():
            warnings.simplefilter('ignore')
            df = pd.read_excel(filename, skiprows=2, decimal=',')

        # extract data columns
        q_df = df[['Datum', 'Durchfluss (Q) m³/s']].copy()
        q_df.columns = ['date', 'q']
        q_df['flag'] = np.NaN
        
        w_df = df[['Datum', 'Wasserstand (W) cm']].copy()
        w_df.columns = ['date', 'w']
        w_df['flag'] = np.NaN
        
        # append
        q.append(q_df)
        w.append(w_df)

        # metadata - get first three columns
        m = dict()
        for i in range(3):
            # these columns need to be unique
            if np.unique(df[df.columns[i]].values).size > 1:
                warnings.warn(f"Column {df.columns[i]} of file {filename} is expected to be unique")
                m = None
                break
            else:
                # add metadata
                m[df.columns[i]] = str(df.iloc[0, i])
        
        # add other stuff
        if m is not None:     
            m['unit_q'] = 'm³/s'
            m['unit_w'] = 'cm'
        meta.append(m)
    
print(f"Parsed {len(meta)} files")

  0%|          | 0/282 [00:00<?, ?it/s]

100%|██████████| 282/282 [05:21<00:00,  1.14s/it]

Parsed 282 files


### Create metadta

this should be straightforward

In [7]:
metadata = pd.DataFrame(meta)
metadata

,Pegelkennziffer,Pegelname,Gewaesser,unit_q,unit_w
0,568350,Rothenthal,Natzschung (Nacetinsky potok),m³/s,cm
1,660001,Hartau 1,Lausitzer Neiße,m³/s,cm
2,567450,Lichtenwalde,Zschopau,m³/s,cm
3,567420,Hopfgarten,Zschopau,m³/s,cm
4,564610,Jahnsdorf,Würschnitz,m³/s,cm
...,...,...,...,...,...
277,563745,Johanngeorgenstadt 4,Schwarzwasser,m³/s,cm
278,582000,Ebersbach,Spree,m³/s,cm
279,564201,Niedermülsen 1,Mülsenbach,m³/s,cm
280,552190,Nebitzschen,Döllnitz,m³/s,cm


In [8]:
id_column = 'Pegelkennziffer'

### Finally run

Now, the Q and W data can be extracted. The cool thing is, that all the id creation, data creation, merging and the mapping from our ids to the original ids and files is done by the context. This is helpful, as we less likely screw something up.

In [9]:
with Bundesland('Sachsen') as bl:
    # save the metadata
    bl.save_raw_metadata(metadata, id_column, overwrite=True)

    # for reference, call the nuts-mapping as table
    nuts_map = bl.nuts_table
    print(nuts_map.head())

    # go for each    
    for m, q_df, w_df in tqdm(zip(meta, q, w), total=len(meta)):
        
        if m is not None:
            # get the provider id
            provider_id = str(m[id_column])
            bl.save_timeseries(q_df, provider_id)
            bl.save_timeseries(w_df, provider_id)

    # check if there were warnings (there are warnings)
    if len(warns) > 0:
        log_path = bl.save_warnings(warns)
        print(f"There were warnings during the processing. The log can be found at: {log_path}")


    nuts_id provider_id                              path
0  DED10000      568350  ./DED/DED10000/DED10000_data.csv
1  DED10010      660001  ./DED/DED10010/DED10010_data.csv
2  DED10020      567450  ./DED/DED10020/DED10020_data.csv
3  DED10030      567420  ./DED/DED10030/DED10030_data.csv
4  DED10040      564610  ./DED/DED10040/DED10040_data.csv


  0%|          | 0/282 [00:00<?, ?it/s]

100%|██████████| 282/282 [00:24<00:00, 11.49it/s]
